In [1]:
import pandas as pd
import win32com.client
import os

In [2]:
#もし発言録まとめの列名が変更された時のために
KOUTEI_ID = "koutei_ID"
SAGYO_ID = "sagyo_ID"
KOUDOU_ID = "koudou_ID"
KOUTEI = "工程"
SAGYO = "主な作業内容"
#列を指定する際にループで増加させるインデックスの増加定数
INDEX_PLUS = 5

In [3]:
file_name = "中間報告用発言録まとめ.xlsx"
sheet_names = pd.ExcelFile(file_name).sheet_names
excels = pd.read_excel(file_name, sheet_name=sheet_names)

In [65]:
skill_table = excels["101"]
skill_table["具体的な行動の仕方"]

0         やらずを見る。
1    やらずの横のバーを見る。
2         消火栓を見る。
3          鉄塔を見る。
4        接地抵抗を見る。
5          親綱を見る。
6                
7          親綱を見る。
Name: 具体的な行動の仕方, dtype: object

In [64]:
if sum(skill_table["具体的な行動の仕方"].isnull()) > 0:
    skill_table["具体的な行動の仕方"] = skill_table["具体的な行動の仕方"].fillna("")
if sum(skill_table["行動のポイント"].isnull()) > 0:
    skill_table["行動のポイント"] = skill_table["行動のポイント"].fillna("")
if sum(skill_table["ポイントの背景"].isnull()) > 0:
    skill_table["ポイントの背景"] = skill_table["ポイントの背景"].fillna("")

In [4]:
xlApp = win32com.client.Dispatch("Excel.Application")

In [10]:
colmuns = [KOUTEI_ID, SAGYO_ID, KOUDOU_ID, KOUTEI, SAGYO]
IDs = pd.DataFrame(index=[], columns=colmuns)
for sheet_name in sheet_names:
    excel = excels[sheet_name]
    excel[KOUTEI_ID] = excel[KOUTEI_ID].fillna(method="ffill").astype(int)
    excel[SAGYO_ID] = excel[SAGYO_ID].fillna(method="ffill").astype(int)
    excel[KOUDOU_ID] = excel[KOUDOU_ID].fillna(method="ffill").astype(int)
    excel[KOUTEI] = excel[KOUTEI].fillna(method="ffill")
    excel[SAGYO] = excel[SAGYO].fillna(method="ffill")
    sub_IDs = excel[colmuns]
    IDs = IDs.append(sub_IDs)
IDs = IDs.drop_duplicates().sort_values([KOUTEI_ID,SAGYO_ID,KOUDOU_ID]).reset_index(drop=True)
IDs

,koutei_ID,sagyo_ID,koudou_ID,工程,主な作業内容
0,1,1,1,足場外,やらず
1,1,1,2,足場外,やらず
2,1,2,1,足場外,消火栓
3,1,3,1,足場外,送電線
4,1,3,2,足場外,送電線
5,1,4,1,足場外,電気盤
6,1,5,1,足場外,大筋交い
7,1,6,1,足場外,溜水
8,2,1,1,頂上,安全帯
9,2,1,2,頂上,安全帯


In [33]:
wb = xlApp.Workbooks.Add()
ws = wb.sheets[0]
ws.Name = "比較技能分析表"
ws.Cells.Font.Name = "メイリオ"
ws.Cells.Font.size = 10
ws.Cells.WrapText = True
xlApp.Visible = True
xlApp.DisplayAlerts = False

In [34]:
#header作成
header = [
    "工程",
    "作業",
    "具体的な行動\n共通人数",
    "行動のポイント\n共通人数",
    "行動の背景\n共通人数",
    "比較"
]
ws.Range(ws.Cells(2,2),ws.Cells(2,7)).Value = header
ws.Range(ws.Cells(2,2),ws.Cells(2,7)).HorizontalAlignment = 3
ws.Range("D2:F2").ColumnWidth = 12
for i, sheet_name in enumerate(sheet_names):
    i += 1 #excelの列インデックスに使うため
    ws.Cells(2,i*3+INDEX_PLUS).Value = sheet_name
    target_range = ws.Range(ws.Cells(2,i*3+INDEX_PLUS),ws.Cells(2,i*3+INDEX_PLUS+2))
    target_range.MergeCells = True
    target_range.HorizontalAlignment = 3
    target_range.ColumnWidth = 30

In [35]:
#技能分析表作成
koutei_ID = 1
sagyo_ID = 1
koutei_st = 3
sagyo_st = 3
img_size = 5#画像のheight(cm)

#はじめの作業、工程列を記入
ws.Cells(koutei_st,2).Value = IDs[KOUTEI][0]
ws.Cells(sagyo_st,3).value = IDs[SAGYO][0]

for row in range(len(IDs)):
    for sub_cnt, sheet_name in enumerate(sheet_names):
        sub_cnt += 1 #excelの列インデックスに使うため
        excel = excels[sheet_name]
        #koutei_ID,sagyou_ID,Koudou_IDが全て一致するものを抽出
        skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][row]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][row]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][row])
        ]
        
        if sum(skill_table["具体的な行動の仕方"].isnull()) > 0:
            skill_table["具体的な行動の仕方"] = skill_table["具体的な行動の仕方"].fillna("")
        if sum(skill_table["行動のポイント"].isnull()) > 0:
            skill_table["行動のポイント"] = skill_table["行動のポイント"].fillna("")
        if sum(skill_table["ポイントの背景"].isnull()) > 0:
            skill_table["ポイントの背景"] = skill_table["ポイントの背景"].fillna("")
        
        """#sagyo_idが変わった時に作業列セル結合し、sagyo_stを更新
        if sagyo_ID != IDs[SAGYO_ID][row]:
            #セル結合
            target_range = ws.Range(
                ws.Cells(sagyo_st,3),
                ws.Cells(ws.UsedRange.Rows.Count+1,3)
            )
            target_range.MergeCells = True
            target_range.HorizontalAlignment = 3
            #sagyo_st更新
            sagyo_st = ws.UsedRange.Rows.Count + 2
            print("row:",row)
            print("changed",sagyo_st)
            #sagyo_ID更新
            sagyo_ID = IDs[SAGYO_ID][row]
            #次の作業記入
            ws.Cells(sagyo_st,3).value = IDs[SAGYO][row]
            
        #koutei_IDが変わった時に工程列をセル結合し、工程を記入
        if koutei_ID != IDs[KOUTEI_ID][row]:
            #工程列をセル結合
            print("row:",row)
            print("mergecell::",koutei_st,":",sagyo_st-1)
            target_range = ws.Range(
                ws.Cells(koutei_st,2),
                ws.Cells(sagyo_st-1,2)#sagyo_stが先に更新されているため-1前の行
            )
            target_range.MergeCells = True
            target_range.HorizontalAlignment = 3
            #koutei_st更新　sagyo_stと同じ（工程が変わるとき作業も変わる）<-変わらないこともあるそこでおかしい
            koutei_st = sagyo_st
            #koutei_ID更新
            koutei_ID = IDs[KOUTEI_ID][row]
            #次の工程記入
            ws.Cells(koutei_st,2).Value = IDs[KOUTEI][row]
           """ 
        #動画があれば、リンクを挿入
        df_video = skill_table.filter(like = "video", axis = 1).reset_index(drop=True)
        if(not(df_video.empty)):
            df_video = df_video.dropna(how="all", axis=1)
            for video_cnt,video_index in enumerate(df_video):
                video_path = os.getcwd() + "/videos/" + df_video[video_index][0]
                #ディレクトリにファイルが存在する場合はリンクを挿入
                if os.path.exists(video_path):
                    ws.Cells(sagyo_st, sub_cnt*3+5+video_cnt).Value = df_video[video_index][0]
                    ws.Cells(sagyo_st, sub_cnt*3+5+video_cnt).HorizontalAlignment = 3
                    ws.Hyperlinks.Add(
                        Anchor = ws.Cells(sagyo_st, sub_cnt*3+INDEX_PLUS+video_cnt), 
                        Address = video_path
                    )
                else:
                    print(video_path, "なんてないで！！")
                    
        #画像があれば挿入
        df_image = skill_table.filter(like = "image", axis = 1)
        if(not(df_image.empty)):
            df_image = df_image.dropna(how="all", axis=1).reset_index(drop=True)
            image_row = sagyo_st + 1#画像を挿入する行数
            #画像行の行高さを調整 img_size*33
            ws.Rows(image_row).RowHeight = img_size * 33 
            for image_cnt,image_index in enumerate(df_image):
                image_path = os.getcwd() + "/images/" + df_image[image_index][0]
                #画像3枚ごとに次の行に挿入
                if image_cnt / 3 == 1:
                    image_row += 1
                    #画像行の行高さを調整 img_size*33
                    ws.Rows(image_row).RowHeight = img_size * 33
                #ディレクトリにファイルが存在する場合は画像を挿入
                if os.path.exists(image_path):
                    image = ws.Shapes.AddPicture(
                        Filename = os.getcwd() + "/images/" + df_image[image_index][0],
                        LinkToFile = False,
                        SaveWithDocument = True,
                        Left = ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).Left,
                        Top = ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).Top,
                        Width = -1,
                        Height= -1
                    )
                    #画像番号入力
                    ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).value = image_cnt + 1
                    ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).HorizontalAlignment = 2
                    ws.Cells(image_row, sub_cnt*3+INDEX_PLUS+image_cnt).VerticalAlignment = 1
                    
                    #画像サイズ調整
                    image.Height = 28.34646 * img_size
                    if image.Width > 28.34646 * 6:#img_sizeで指定した幅(cm)のアスペクト比計算で幅が6cmを超える場合は7cmに設定
                        image.Width = 28.34646 * 6
                    
                    #画像をセルの中央へ配置
                    image.Left = ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Left \
                        + (ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Width - image.Width) / 2
                    image.Top = ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Top\
                        + (ws.Cells(image_row,sub_cnt*3+INDEX_PLUS+image_cnt).Height - image.Height) / 2
                else:
                    print(image_path, "なんてないで！！")
    
    #koudou_idが1の時はミニヘッダーを挿入       
    if IDs[KOUDOU_ID][row] == 1:
        #miniheader記入
        minihead_row = ws.UsedRange.Rows.Count + 2
        miniheader = ["具体的な行動", "行動のポイント", "行動の背景"]
        for i in range(len(sheet_names)):
            i += 1 #excelの列インデックスに使うため
            target_range = ws.Range(
                ws.Cells(
                    minihead_row,
                    i*3+INDEX_PLUS
                ),
                ws.Cells(
                    minihead_row,
                    i*3+INDEX_PLUS+2)
            )
            target_range.Value = miniheader
            target_range.HorizontalAlignment = 3
            
    #技能分析表の内容を記入
    skill_table_row = minihead_row + 1
    #一人ずつ内容を埋めていく
    for sub_cnt, sheet_name in enumerate(sheet_names):
        sub_cnt += 1 #excelの列インデックスに使うため
        excel = excels[sheet_name]
        #koutei_ID,sagyou_ID,Koudou_IDが全て一致するものを抽出
        skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][row]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][row]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][row])
        ]
        #内容記載に必要部分のみ抽出
        skill_table = skill_table.loc[:, ["具体的な行動の仕方", "行動のポイント", "ポイントの背景"]]
        #行動の行数
        num_koudou = skill_table.shape[0]
        #内容を埋めていく
        if num_koudou == 1:
            ws.Range(
                ws.Cells(
                    skill_table_row,
                    sub_cnt*3+INDEX_PLUS
                ),
                ws.Cells(
                    skill_table_row,
                    sub_cnt*3+INDEX_PLUS+2)
            ).value = skill_table.values.tolist()
        else:
            ws.Range(
                ws.Cells(
                    skill_table_row,
                    sub_cnt*3+INDEX_PLUS
                ),
                ws.Cells(
                    skill_table_row+num_koudou,
                    sub_cnt*3+INDEX_PLUS+2)
            ).value = skill_table.values.tolist()
            

<ipython-input-35-dc7eeed8cdcb>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_table["ポイントの背景"] = skill_table["ポイントの背景"].fillna("")
<ipython-input-35-dc7eeed8cdcb>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skill_table["行動のポイント"] = skill_table["行動のポイント"].fillna("")
<ipython-input-35-dc7eeed8cdcb>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

C:\Users\ts1697\OneDrive - Tobii AB (Publ)\Documents\Python Scripts\比較技能分析表/images/102_7_1_ashiba.jpg なんてないで！！


In [50]:
skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][0]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][0]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][0])
        ]
if sum(skill_table["具体的な行動の仕方"].isnull()) > 0:
    skill_table["具体的な行動の仕方"] = skill_table["具体的な行動の仕方"].fillna("")
skill_table

,その他メモ,時間,koutei_ID,工程,sagyo_ID,主な作業内容,koudou_ID,具体的な行動の仕方,行動のポイント,ポイントの背景,image_1,image_2,image_3,image_4,video_1,video_2,video_3,video_4
0,NaN,00:04:18.224000,1,足場外,1,やらず,1,やらずを見る。,全体にやらずがあるか確認する。,やらずがないと転倒してしまうので。,101_1_1_yarazu.jpg,101_1_2_yarazu.jpg,NaN,NaN,101_1_1_yarazu.mp4,101_1_2_yarazu.mp4,NaN,NaN


In [76]:
skill_table = excel[
            (excel[KOUTEI_ID] == IDs[KOUTEI_ID][0]) &
            (excel[SAGYO_ID] == IDs[SAGYO_ID][0]) &
            (excel[KOUDOU_ID] == IDs[KOUDOU_ID][0])
        ]
df_image = skill_table.filter(like = "image", axis = 1)
df_image = df_image.dropna(how="all", axis=1).reset_index(drop=True)
skill_table = skill_table.loc[:,["具体的な行動の仕方","行動のポイント","ポイントの背景"]]
ws.Range("D3:F3").value = skill_table.iloc[0,:]

In [12]:
xlApp.Quit()